<div class="alert block alert-info alert">

# <center> Scientific Programming in Python

## <center>Karl N. Kirschner<br>Bonn-Rhein-Sieg University of Applied Sciences<br>Sankt Augustin, Germany

# <center> Testing Inside your Code<br><br>and<br><br>Testing the Code Itself

<!-- <br><br> -->

<hr style="border:2px solid gray"></hr>

**Note**: All user-defined functions in the notebook do not include document strings (i.e. block comments) or internal checks. This is purposely done to focus on the teaching aspects of the lecture. **A full and proper user-defined function would include these.**

In [ ]:
## For extra information given within the lectures

from IPython.display import HTML


def set_code_background(color: str):
    ''' Set the background color for code cells.

        Source: psychemedia via https://stackoverflow.com/questions/49429585/
                how-to-change-the-background-color-of-a-single-cell-in-a-jupyter-notebook-jupy

        To match Jupyter's dev class colors:
            "alert alert-block alert-warning" = #fcf8e3

        Args:
            color: HTML color, rgba, hex
    '''

    script = ("var cell = this.closest('.code_cell');"
              "var editor = cell.querySelector('.input_area');"
              f"editor.style.background='{color}';"
              "this.parentNode.removeChild(this)")
    display(HTML(f'<img src onerror="{script}">'))


set_code_background(color='#fcf8e3')

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

**Sidenote**:  A quick reminder about typing

Note that **typing** (a.k.a. type hinting, annotating functions) - actually **doesn't enforce** anything.

We use them for our own and others' **clarification** of the code and its usage.

In [ ]:
def user_function(number: float) -> float:
    return number*2

In [ ]:
user_function(number='me')

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

<hr style="border:1px solid gray"></hr>

# Testing Inside Your Code


## `assert` versus `raise`

### `assert`

**Usage**: `assert test_condition, 'Error message to display'`

- specify expectations for what your variables are
- A helpful way to <font color='Red'>**debug**</font> code


- Includes **traceback** (a.k.a. stack trace, stack traceback, backtrace) to show you the sequence of calls and associated problems
    - https://realpython.com/python-traceback

- An `assert` statement is **equivalent** to the following code:

```python
if not condition:
    raise AssertionError()
```


- <font color='DodgerBlue'>Asserts are **not** meant to **test for expected conditions**</font>
    - **Security issue**: see below.

In [ ]:
my_test_obj = 5

assert my_test_obj != 5, 'ERROR MESSAGE FROM YOUR INSTRUCTOR - I LIKE YELLING WHEN I TYPE MESSAGES'

### Practical usage

1. Create & demo a simple function without and `assert` statement
2. Create & demo one with an `assert` statement

In [ ]:
def divide_me(number_1: float, number_2: float) -> float:

    return number_1/number_2


divide_me(number_1=1.0, number_2=2.0)

The functions <font color='DodgerBlue'>**runs fine**</font>, **but** we could provide more customized feedback.

In [ ]:
divide_me(number_1=1.0, number_2=None)

An `assert` can check to make sure that a variable is not `None` (via `!=` or `is not None`), allowing you to <font color='DodgerBlue'>customize the feedback (i.e. placing the error into context)</font>.

In [ ]:
def divide_me(number_1: float, number_2: float) -> float:

    assert number_1 is not None, 'The numerator was not provided.'
    assert number_2 != None, 'The denominator was not provided.'
    assert number_2 != 0.0, "Error: you can't divide by zero. How dare you try!"

    return number_1/number_2


divide_me(number_1=1.0, number_2=None)

### `Assert` Security issue

There is also a way for users to **circumvent** (i.e. get around) assert statements.

From a bash terminal:
- **python assert_example.py**: reads the assert statement (everything seems to be working properly)
- **python -O assert_example.py**: the assert statement is not read (i.e. it is bypassed), and instead prints a standard error statement

In the following, I will also demonstrate how one can <font color='DodgerBlue'>write a Python script in Jupyter notebook</font> and save it to your hard drive.

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

**Sidenote**: Jupyter Notebook <font color='DodgerBlue'>Magic Commands</font>

https://ipython.readthedocs.io/en/stable/interactive/magics.html

https://towardsdatascience.com/useful-ipython-magic-commands-245e6c024711

1. line magic commands (`%`) - the input directly follows the `%`
2. cell magic commands (`%%`) - the entire cell becomes the input

- `%load` import code from a Python script (e.g. `%load filename.py`)

- `%%writefile filename`: write the contents of a cell to a file

- `%timeit` and `%%timeit`: code performance

In [ ]:
%lsmagic

Getting help is easy - just add a `?` to the end of the command:

In [ ]:
%%writefile?

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

In [ ]:
%%writefile 'assert_example.py'
#!/usr/bin/env python

'''
An example of why using assert to test for expected conditions is bad.

You can "turn off" asserts by typing "python -O filename.py" and thus
    bypassing the check.

Expectations when running the code in a bash terminal:
python assert_example.py -> assert is read and prints out its error
python -O assert_example.py -> assert is not read and the program runs
'''

def simple_print(number_1: float, number_2: float) -> float:

    assert number_1 != None, "Error: you did not provide a numerator"
    assert number_2 != None, "Error: you did not provide a denominator"
    assert number_2 != 0, "Number 2 can't be zero"

    print(number_1, number_2)

simple_print(number_1=None, number_2=0.0)

In [ ]:
%cat 'assert_example.py'

**Note:** The following works when executed from a **local computer** that has the above python script saved to the working directory.

In [ ]:
! python3 assert_example.py

Including a **`-O`** will <font color='DodgerBlue'>**ignore all assert statements**</font>.

Why do this?
- Also sets the special builtin name `__debug__` to `False` (`True` by default)
    - E.g.: `if __debug__ then:`
- Useful while working/optimizing code.
- Results is a very small performance boost.

In [ ]:
! python3 -O assert_example.py

In [ ]:
%rm assert_example.py

#### Take-home message about `asserts` and in-my-opinion:

- They help debug your code during its development.
- They are not as robust as one thinks.
- There are better ways to have internal checks.

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

## `isinstance` - another way to provide internal checks in your code

https://docs.python.org/3/library/functions.html#isinstance

- this is a python built-in function
- can check on a variable's type (e.g., int, float, str)

<hr style="border:1.5px dashed gray"></hr>

# Exceptions: testing for expected conditions
<!-- EAFP ("<font color='DodgerBlue'>**E**asier to **A**sk for **F**orgiveness than **P**ermission</font>")
- often adopted by programmers,
- but is that good practice?

EAFP can be implemented via the: -->
### `try`-`except` statement
- tells your code to try something
- then tell it what to do if it fails based on an exception type

 
#### Strengths:
1. your code will <font color='DodgerBlue'>**continue**</font> when it encounters a problem<br><br>

2. <font color='DodgerBlue'>**faster** than if statements</font> for when <font color='DodgerBlue'>majority of the planned tasks are **expected** to be **successful**</font> (i.e., they don't encounter an exception)

<br><br>
**Simple Example**

In [ ]:
try:
    print(5/0)
except ZeroDivisionError:
    print("Error: You can't have a zero in the denominator.")

**More Sophisticated Example**

Let's set up a <font color='DodgerBlue'>division calculator</font> that allows users to input numbers and quit at any time using while and if loops (to demonstrate via a comparison of code).

1. First, set something up without `try`-`except` in order to see its advantage later.
2. Second, do the same thing with `try`-`except`

<font color='DodgerBlue'>Demonstrate the following</font>:
1. normal operation
2. exiting by typing 'q'
3. <font color='Red'>O</font> (i.e., a capital alphabet letter "O", as in "O"ktoberfest (also demonstrates traceback error))

Without `try`-`except` statement:

In [ ]:
## print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator = ')
    if numerator == 'q':
        break

    denominator = input('Denominator = ')
    if denominator == 'q':
        break

    if denominator == '0':
        print("You can't have a zero in the denominator.")
        break

    answer = float(numerator)/float(denominator)
    print(f'Answer for {numerator}/{denominator} = {answer}\n')

Modify the above code to use a **`try`-`except` statement**, and try it with <font color='Red'>O</font>.

**Note**: <font color='DodgerBlue'>Multiple `except` conditions via a **tuple**</font>:<br>
`except (ZeroDivisionError, ValueError):`
- `ZeroDivisionError` when the denominator is zero
- `ValueError` for when a string is given as an input

In [ ]:
print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator = ')
    if numerator == 'q':
        break

    denominator = input('Denominator = ')
    if denominator == 'q':
        break

    try:
        answer = float(numerator)/float(denominator)
        print(f'Answer for {numerator}/{denominator} = {answer}\n')

    except (ZeroDivisionError, ValueError):
        print('Your input was either not a number, or you are dividing by a zero.')

<font color='DodgerBlue'>Now the code continues to run, even though an error was raised!</font>

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

**Sidenote**: `TypeError` versus `ValueError`

- `ValueError`: raised when an operation or function receives an argument that
    1. has the **correct type**, but
    2. an **inappropriate value**


- `TypeError`: raised when passing arguments of the **wrong type** (e.g. passing a `list` when an `int` is expected)

Best understood through the following example...

In [ ]:
def attempt_float(number):
    try:
        return float(number)

    except (TypeError):
        print(f'Input ({number}) (type: {type(number)}) was not the right type (i.e TypeError).')

    except (ValueError):
        print(f'Input ({number}) (type: {type(number)}) was not a correct value (i.e. ValueError).')

In [ ]:
attempt_float(0.1)

<div class="alert alert-block alert-warning">

The built-in float can also accept a string if it is a decimal:
"If the argument is a string, it should contain a decimal number..." (https://docs.python.org/3/library/functions.html#float)

In [ ]:
attempt_float('0.1')

In [ ]:
## a ValueError (correct type, wrong value)
attempt_float('something')

In [ ]:
## a TypeError (wrong type)
attempt_float([0.1, 0.2])

<div class="alert alert-block alert-warning">
<hr style="border:1.5px dashed gray"></hr>

<hr style="border:2px solid gray"></hr>

<h1 align='center'>Test Driven Development
    
<h2 align='center'> a.k.a. Unit Tests

https://docs.python.org/3/library/unittest.html

## Test Driven Development - writing tests before you write your production code
1. Ensures proper and directed functionality of your code
 - creating **concise** code that does a **single** thing (e.g., user-defined functions)
2. Helps you plan your code - what do you **actually want** to do (critical thinking)
3. Reduces **errors**
4. Ensures **reproducibility**
5. Helps to ensure a code's **long life**

## The Workflow Concept
1. Write a failing test
2. Run and ensure failure
3. Write code to pass
4. Run and ensure passing
5. Refactor (i.e., restructure/clean up code without changing it final result)
6. Redo steps 1-5

## Scientific and Data Research
It is **CRITICAL** that:
1. you get the correct results
2. you make it generate reproducible results, especially as the code becomes bigger (and changes)

#### Assert statements that can be used in unittest library

https://docs.python.org/3/library/unittest.html#module-unittest


| Method | Checks that | New in |
|:------|:-:|:-:|
| assertEqual(a, b) | a == b | |
| assertNotEqual(a, b)| a != b | |
| assertTrue(x) | bool(x) is True | |
| assertFalse(x) | bool(x) is False | |
| assertIs(a, b) | a is b | 3.1 |
| assertIsNot(a, b) | a is not b | 3.1 |
| assertIsNone(x) | x is None | 3.1 |
| assertIsNotNone(x) | x is not None | 3.1 |
| assertIn(a, b) | a in b | 3.1 |
| assertNotIn(a, b) | a not in b | 3.1 |
| assertIsInstance(a, b) | isinstance(a, b) | 3.2 |
| assertNotIsInstance(a, b) | not isinstance(a, b) | 3.2 |
| | | |
| | | |
| assertAlmostEqual(a, b) | round(a-b, 7) == 0 | |
| assertNotAlmostEqual(a, b) | round(a-b, 7) != 0 | |
| assertGreater(a, b) | a > b | 3.1 |
| assertGreaterEqual(a, b) | a >= b | 3.1 |
| assertLess(a, b) | a < b | 3.1 |
| assertLessEqual(a, b) | a <= b | 3.1 |
| assertRegex(s, r) | r.search(s) | 3.1 |
| assertNotRegex(s, r) | not r.search(s) | 3.2 |
| assertCountEqual(a, b) | a and b have the same elements in the same number, regardless of their order. | 3.2 |


| Method | Used to compare | New in|
|:------|:-:|:-:|
| assertMultiLineEqual(a, b) | strings | 3.1 |
| assertSequenceEqual(a, b) | sequences | 3.1 |
| assertListEqual(a, b) | lists | 3.1 |
| assertTupleEqual(a, b) | tuples | 3.1 |
| assertSetEqual(a, b) | sets or frozensets | 3.1 |
| assertDictEqual(a, b) | dicts | 3.1 |

Demonstrate the following two scenarios:
1. Scenario 1: the unit test runs with everything correct
2. Scenario 2: the unit test runs, but with errors
     - A new `assertEqual` is added

**Note**: We will include <font color='DodgerBlue'>additional assert statements just to demonstrate how the output of a unit test looks like</font>, even though it is not relevant to our user-defined function.

**Scenario 1**: the unit test runs with everything <font color='DodgerBlue'>correctly</font>

Define a user-defined function to demo how that is done:

In [ ]:
def hello_world():
    return 'hello world'

**Note**: `isupper()`

- `str.isupper()`: Return `True` if all characters in a given string are uppercase, otherwise it is `False`.
    
    - https://docs.python.org/3/library/stdtypes.html

In [ ]:
import unittest


class MyFirstUniTTests(unittest.TestCase):

    def test_isEqual(self):
        self.assertEqual(hello_world(), 'hello world')

    def test_isLess(self):
        self.assertLess(5, 10)

    def test_isLessEqual(self):
        self.assertLessEqual(10, 10)

    def test_isUpperTrue(self):
        self.assertTrue('FOO'.isupper())

    def test_isUpperFalse(self):
        self.assertFalse('Foo'.isupper())


## Normal usage (in a .py script)
#if __name__ == '__main__':
#    unittest.main()

## For usage in jupyter and colaboratory (due to the kernel)
if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

**Scenario 2**: the unit test runs, but some <font color='red'>errors</font> occur

In [ ]:
class MyFirstUniTTests(unittest.TestCase):

    def test_fail(self):
       self.assertEqual(hello_world(), 'bye world')

    def test_isEqual(self):
        self.assertEqual(hello_world(), 'hello world')

    def test_isLess(self):
        self.assertLess(5, 10)

    def test_isLessEqual(self):
        self.assertLessEqual(10, 10)

    def test_isUpperTrue(self):
        self.assertTrue('FOO'.isupper())

    def test_isUpperFalse(self):
        self.assertFalse('Foo'.isupper())


if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

## PyTest

1. A command-line (e.g., using a bash shell) driven testing approach
2. Simplifies and helps organize unit tests
    - done by creating **user-defined functions** for **each test** that you want to do

https://docs.pytest.org/en/7.1.x/contents.html#

In [ ]:
%%writefile test_sum.py
''' The following will be created:
        1. Four unit test functions
            a. First 3 will pass
            b. Last 1 will fail
'''

def test_pass_add_list_1():
    ''' 1st unit test
    '''
    test_list = [1, 2, 3, 4]
    assert sum(test_list) == 10


def test_pass_add_list_2():
    ''' 2nd unit test
    '''
    test_list = [1, 2, 3, 4, 5]
    assert sum(test_list) == 15


def test_pass_add_list_3():
    ''' 3rd unit test
    '''
    test_list = [1, 2, 3, 4, 5, 6]
    assert sum(test_list) == 21


def test_fail_add_list_4():
    ''' 4th unit test
        Should Fail
    '''
    print('PRINT STATEMENT FOR FAILING TEST FUNCTION.')
    
    test_list = [1, 2, 3, 4, 5, 6]
    assert sum(test_list) == 0

PyTest will give the following output:
- `.` (dot) = test <font color='DodgerBlue'>passed</font>
- `F` =  test has <font color='DodgerBlue'>failed</font>
- `E` =  test raised an <font color='DodgerBlue'>unexpected exception</font>

In [ ]:
! pytest test_sum.py

**Output**
1. The first three test passed
2. The fourth test fails
3. A traceback is given concerning the error
4. None of the print statements are seen

<font color='DodgerBlue'>To see print commands</font> within the user-defined functions, <font color='DodgerBlue'>use `-s` option</font>:

(`-s` is a shortcut for `--capture=no` - see `pytest --help`)

In [ ]:
! pytest test_sum.py -s

Clean up directory:

In [ ]:
%rm test_sum.py

## Example of Notebook function and PyTesting

Create a User-defined Function

In [ ]:
def calculate_area(length, width):
    """ Calculate the area of a rectangle.
    """
    if length <= 0 or width <= 0:
        raise ValueError("Dimensions must be positive.")
    return length * width

Create PyTest

In [ ]:
%%writefile test_calculate_area.py
import pytest

def calculate_area(length, width):
    """ Calculate the area of a rectangle.
    """
    if length <= 0 or width <= 0:
        raise ValueError("Dimensions must be positive.")
    return length * width


def test_positive_dimensions():
    # Test case with standard positive inputs
    assert calculate_area(5, 4) == 20


def test_square_dimensions():
    # Test case for a square (equal sides)
    assert calculate_area(10, 10) == 100


def test_zero_or_negative_dimensions():
    # Test case for invalid input using pytest.raises
    with pytest.raises(ValueError):
        calculate_area(-1, 5)
    with pytest.raises(ValueError):
        calculate_area(0, 5)

In [ ]:
! pytest test_calculate_area.py

In [ ]:
%rm test_calculate_area.py

## Introducing `pytest.mark.parametrize`

In [ ]:
%%writefile test_even.py
import pytest


def add(number_1: int|float, number_2: int|float) -> float:
    """
    Returns the sum of two numbers.

    Parameters:
        number_1: The first number.
        number_2: The second number.

    Returns:
        int or float: The sum of a and b.
    """
    return number_1 + number_2


def test_add_multiple_inputs():
    ''' An initial attempt that uses a for loop within the test (avoid this).
    '''
    test_data = [(1, 2, 3),
                 (-1, -1, -2),
                 (5, 0, 5)]
    for num_1, num_2, expected in test_data:
        assert add(number_1=num_1, number_2=num_2) == expected

In [ ]:
!pytest -vs test_even.py

In [ ]:
%%writefile test_add_numbers.py
import pytest


def add(number_1: int|float, number_2: int|float) -> float:
    """
    Returns the sum of two numbers.

    Parameters:
        number_1: The first number.
        number_2: The second number.

    Returns:
        int or float: The sum of a and b.
    """
    return number_1 + number_2


@pytest.mark.parametrize("num_1, num_2, expected", [(1, 2, 3),        # Test Case 1: Positive numbers
                                                    (-1, -1, -2),     # Test Case 2: Negative numbers
                                                    (5, 0, 5),        # Test Case 3: Testing with zero
                                                    (100, -50, 50)])  # Test Case 4: Positive and negative

def test_add(num_1, num_2, expected):
    assert add(number_1=num_1, number_2=num_2) == expected

In [ ]:
!pytest -vs test_add_numbers.py

In [ ]:
%rm test_add_numbers.py